# RAG Usando o Serviço de Agente Azure AI e o Semantic Kernel

Este exemplo de código demonstra como criar e gerir um agente Azure AI para geração aumentada por recuperação (RAG) utilizando o `Serviço de Agente Azure AI` e o `Semantic Kernel`. O agente processa as consultas dos utilizadores com base no contexto recuperado e fornece respostas precisas em conformidade.


Correção da Versão do SQLite  
Se encontrar o erro:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Descomente este bloco de código no início do seu notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importar Pacotes
O código abaixo importa os pacotes necessários:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Geração Aumentada por Recuperação com Semantic Kernel & Azure AI Agent Service

Este exemplo demonstra como utilizar o **Azure AI Agent Service** para realizar **Geração Aumentada por Recuperação (RAG)**, combinando um modelo de linguagem com contexto específico de domínio proveniente de um documento carregado.

### Como Funciona

1. **Carregamento de Documento**: Um ficheiro markdown (document.md) contendo informações (política de seguro de viagem da Contoso) é carregado para o serviço de agente.

2. **Criação de Armazenamento Vetorial**: O documento é indexado num armazenamento vetorial para permitir pesquisa semântica sobre o seu conteúdo.

3. **Configuração do Agente**: Um agente é instanciado utilizando o modelo `gpt-4o` com as seguintes instruções rigorosas:
   - Responder apenas a perguntas com base no conteúdo recuperado do documento.
   - Recusar-se a responder caso a pergunta esteja fora do âmbito.

4. **Integração da Ferramenta de Pesquisa de Ficheiros**: A `FileSearchTool` é registada com o agente, permitindo que o modelo procure e recupere trechos relevantes do documento indexado durante a inferência.

5. **Interação com o Utilizador**: Os utilizadores podem fazer perguntas. Se forem encontradas informações relevantes no documento, o agente gera uma resposta fundamentada.  
   Caso contrário, o agente responde explicitamente que o documento não contém informações suficientes.


### Função Principal

Esta secção descreve a função principal do programa e como ela é utilizada.

#### Descrição

A função principal é o ponto de entrada do programa. É aqui que a execução começa e onde as principais operações são coordenadas.

#### Parâmetros

- `inputData`: Os dados de entrada que a função principal processará.
- `configOptions`: As opções de configuração que determinam o comportamento da função.

#### Retorno

A função principal devolve um objeto que contém os resultados do processamento. Certifique-se de verificar o formato do retorno antes de utilizá-lo.

#### Exemplo de Utilização

```python
result = mainFunction(inputData, configOptions)
print(result)
```

#### Comentários

- Certifique-se de que os dados de entrada estão no formato correto antes de chamar a função.
- Utilize as opções de configuração para personalizar o comportamento da função conforme necessário.

#### Erros Comuns

- Passar dados de entrada inválidos pode causar erros inesperados.
- Não configurar corretamente as opções pode levar a resultados indesejados.

#### Notas Adicionais

- Esta função foi projetada para ser modular e reutilizável em diferentes partes do programa.
- Para mais informações sobre os parâmetros e retorno, consulte a documentação detalhada.


Certifique-se de executar primeiro `az login` usando o Azure CLI para que o contexto de autenticação adequado seja fornecido ao usar o `DefaultAzureCredential`. O Azure AI Agent Service não utiliza chaves de API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Por favor, forneça o conteúdo do ficheiro Markdown que deseja traduzir, e eu farei a tradução seguindo as regras que indicou.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
